In [10]:
import json
import numpy as np
import pickle
import utils
import sys
import os
import math
from collections import defaultdict

In [2]:
try:
    sys.path.append('/opt/MatterSim/build/')  # local docker or Philly
    import MatterSim
except: 
    # local conda env only
    sys.path.append('/home/hoyeung/Documents/vnla/code/build')  
    import MatterSim

In [3]:
PT_DATA_DIR = "/home/hoyeung/blob_matterport3d/"
os.environ['PT_DATA_DIR'] = "/home/hoyeung/blob_matterport3d/"

EXP_DIR = "/home/hoyeung/blob_experiments/"

In [4]:
with open(PT_DATA_DIR + "asknav/asknav_val_seen.json", "r") as f:
    seen_data = json.load(f)
    
len(seen_data)

4874

In [17]:
with open(PT_DATA_DIR + "asknav/asknav_train.json", "r") as f:
    train_data = json.load(f)
    
len(train_data)

94798

In [5]:
res_dir = "output_philly/20200207_philly_rerun_vnla/dagger_no_ask/20200207_philly_rerun_vnla_nav_sample_ask_teacher_val_seen_for_eval.json"
with open(EXP_DIR + res_dir, "r") as f:
    seen_results = json.load(f)
    
len(seen_results)

4874

In [6]:
with open('/home/hoyeung/blob_matterport3d/semantics/asknav_tr_scans.txt', "r") as f:
    asknav_tr_scans = f.read().split('\n')[:-1]
len(asknav_tr_scans)

56

In [7]:
all_panos_to_region = {}
for scan in asknav_tr_scans:
    all_panos_to_region[scan] = utils.load_panos_to_region(scan,"")

In [8]:
viewix_heading_elevation_map = {}
for i in range(0,12):
    viewix_heading_elevation_map[i] = (i*math.pi/6, -math.pi/6)
for i in range(12,24):
    viewix_heading_elevation_map[i] = (i*math.pi/6, 0)
for i in range(24,36):
    viewix_heading_elevation_map[i] = (i*math.pi/6, +math.pi/6)

In [9]:
PT_DATA_DIR = '/home/hoyeung/blob_matterport3d/'
sim = MatterSim.Simulator()
sim.setRenderingEnabled(False)
sim.setDiscretizedViewingAngles(True)
sim.setCameraResolution(640, 480)
sim.setCameraVFOV(math.radians(60))
sim.setNavGraphPath(
    os.path.join(PT_DATA_DIR, 'connectivity'))
sim.init()

### Analysis

In [ ]:
# From seen data, get the most common sequences

# given each starting point image, find the most common gold sequence

In [12]:
seen_data[0].keys()

dict_keys(['scan', 'object_name', 'end_regions', 'end_region_name', 'start_region', 'path_id', 'trajectories', 'heading', 'start_region_name', 'instructions', 'distances', 'object_indices', 'paths'])

In [15]:
seen_data[0]['heading']

3.665191429188092

In [17]:
# same starting point, always same viewix?
seen_results[0].keys()

dict_keys(['agent_ask', 'instr_id', 'trajectory', 'subgoals', 'agent_nav', 'teacher_ask_reason', 'agent_nav_logit_masks', 'scan', 'agent_ask_softmax', 'agent_nav_softmax_final', 'agent_ask_logits', 'agent_nav_softmax_initial', 'teacher_ask', 'agent_nav_logits_initial', 'agent_nav_logits_final', 'agent_ask_logit_masks', 'teacher_nav'])

In [19]:
seen_results[0]['trajectory']

[['fb76fb85a5984f3b871b02607a535c8d', 2.0943951023931953, 0.0],
 ['b28f29d6260f46b6bd9eaa97d787ea53', 2.0943951023931953, 0.0],
 ['b28f29d6260f46b6bd9eaa97d787ea53', 2.617993877991494, 0.0],
 ['b28f29d6260f46b6bd9eaa97d787ea53', 3.141592653589793, 0.0],
 ['aea462d50c1548abb9e01fa60251a4ad', 3.141592653589793, 0.0],
 ['aea462d50c1548abb9e01fa60251a4ad', 3.665191429188092, 0.0],
 ['aea462d50c1548abb9e01fa60251a4ad', 4.1887902047863905, 0.0],
 ['aea462d50c1548abb9e01fa60251a4ad', 4.71238898038469, 0.0],
 ['aea462d50c1548abb9e01fa60251a4ad', 5.235987755982988, 0.0],
 ['aea462d50c1548abb9e01fa60251a4ad', 5.759586531581287, 0.0],
 ['b28f29d6260f46b6bd9eaa97d787ea53', 5.759586531581287, 0.0]]

In [39]:
seen_data[0]['paths'][0][0]

'6cba04f9e76f463094dd2a6ff64d1e9f'

In [33]:
increment_30 = 2 * math.pi / 12.0
3.141592653589793/ increment_30

6.0

In [ ]:
# map [(scan, viewpoint, viewix)] : [full gold trajectories]

In [18]:
full_trajectories_seen = defaultdict(list)

for dat in seen_data:
    scan = dat['scan']
    start_viewpoint = dat['paths'][0][0]
    viewix = dat['heading'] // 12
    for path in dat['paths']:
        full_trajectories_seen[(scan, start_viewpoint, viewix)].append(path)

In [21]:
full_trajectories_train = defaultdict(list)

for dat in train_data:
    scan = dat['scan']
    start_viewpoint = dat['paths'][0][0]
    viewix = dat['heading'] // 12
    for path in dat['paths']:
        full_trajectories_train[(scan, start_viewpoint, viewix)].append(path)

In [32]:
full_trajectories_seen_res = defaultdict(list)

for res in seen_results:
    scan = res['scan']
    start_viewpoint = res['trajectory'][0][0]
    viewix = res['trajectory'][0][1] // 12
    path = [step[0] for step in res['trajectory']]
    full_trajectories_seen_res[(scan, start_viewpoint, viewix)].append(path)

In [23]:
len(full_trajectories_seen)

3012

In [28]:
len(full_trajectories_seen_res)

4646

In [24]:
len(full_trajectories_train)

6774

In [34]:
seen_results_keys = []
for key in full_trajectories_seen_res:
    seen_results_keys.append(key)

In [35]:
seen_results_keys

[('sT4fr6TAbpF', '55abf0a58ae34456a58279523de80df6', 0.0),
 ('pRbA3pwrgk9', 'ec8f8c12b70e4969ae24adb7031d9312', 0.0),
 ('VFuaQ6m2Qom', '5fb269a812f349b4bc2ad5dfac000b85', 0.0),
 ('29hnd4uzFmX', '1c0715389b644f578961d420672e3613', 0.0),
 ('r1Q1Z4BcV1o', '77cfb227d30443428d4c7787a3da713c', 0.0),
 ('759xd9YjKW5', '282d98c53101421f9df5e0aed5d356a3', 0.0),
 ('17DRP5sb8fy', '1a41339ece1846eda6a924cdb4c417dd', 0.0),
 ('5LpN3gDmAk7', '1bc6dad179a6490691f99433c6e45ff6', 0.0),
 ('aayBHfsNo7d', 'c8877195d3ad4ea5a4736bf0ed6979c5', 0.0),
 ('cV4RVeZvu5T', '9afe7c95718743e288c6f3dceb252faf', 0.0),
 ('SN83YJsR3w2', '5226cb9e03da4936a504bdcd86b8fc85', 0.0),
 ('JeFG25nYj2p', '3b725579bc064747a62acc84fde8a41e', 0.0),
 ('b8cTxDM8gDG', 'f2e403f149ae451eb8fc5ab27ec96e9f', 0.0),
 ('vyrNrziPKCB', '880fada214cb4df7bd77378c1d9d6fcd', 0.0),
 ('ac26ZMwG7aT', '6967b846bd9c4c7a9c9c9dfab6d5eb34', 0.0),
 ('JF19kD82Mey', '68e4c4477aa442cbbb9a9f1e54b9b831', 0.0),
 ('29hnd4uzFmX', '42135c66aa4d464aa8a91630c187ed99', 0.0

In [36]:
key = seen_results_keys[1]

In [37]:
full_trajectories_seen[key]

[['ec8f8c12b70e4969ae24adb7031d9312',
  '3851376d4f84494ebdc080b34ddc0f5e',
  '558c07ff6d6d4d268f6c8de7abd76d1d',
  'cb6b9124a9594b4788966d28129cf531',
  '090a8663beeb4ce6aeadfe720ea0c8e9',
  '4d76fa50a0e743e695fb048b4a7766d5',
  'f5a52ba78b2e4772a7315af44e0439e0',
  '2238391e894144e9aaf38aaa16433153',
  '4df525c1410447848d24dedb3c31a7be',
  'f923d8ec1f32400c864837ba572fe199',
  '8b07a4b08cf447abb246769d8dce8494',
  '950f2117a6164fbc8d2faff96c40a1e9'],
 ['ec8f8c12b70e4969ae24adb7031d9312',
  '3851376d4f84494ebdc080b34ddc0f5e',
  '558c07ff6d6d4d268f6c8de7abd76d1d',
  'cb6b9124a9594b4788966d28129cf531',
  '090a8663beeb4ce6aeadfe720ea0c8e9',
  '4d76fa50a0e743e695fb048b4a7766d5',
  'ae8b1eb9dcd444688117d22839834a20',
  'fdd1efb5300a4a57b6537c492c27abe8',
  '0350fc96e88c4a52886d4eb50b2d52c6',
  'c19e6a85e45d4aae9e9a4b703650132b'],
 ['ec8f8c12b70e4969ae24adb7031d9312',
  '3851376d4f84494ebdc080b34ddc0f5e',
  '087bf4ad4a0b4a17afc2d6c173eceff3',
  '99142bab4f864a50ba45e703c5b7f08d',
  '14c83ba

In [38]:
full_trajectories_train[key]

[['ec8f8c12b70e4969ae24adb7031d9312',
  '3851376d4f84494ebdc080b34ddc0f5e',
  '558c07ff6d6d4d268f6c8de7abd76d1d',
  'cb6b9124a9594b4788966d28129cf531',
  '090a8663beeb4ce6aeadfe720ea0c8e9',
  '4d76fa50a0e743e695fb048b4a7766d5',
  'f5a52ba78b2e4772a7315af44e0439e0'],
 ['ec8f8c12b70e4969ae24adb7031d9312',
  '3851376d4f84494ebdc080b34ddc0f5e',
  '087bf4ad4a0b4a17afc2d6c173eceff3',
  '99142bab4f864a50ba45e703c5b7f08d',
  '14c83bad01c34657ab846a7a2446e812'],
 ['ec8f8c12b70e4969ae24adb7031d9312',
  '3851376d4f84494ebdc080b34ddc0f5e',
  '558c07ff6d6d4d268f6c8de7abd76d1d',
  'cb6b9124a9594b4788966d28129cf531',
  '090a8663beeb4ce6aeadfe720ea0c8e9',
  '4d76fa50a0e743e695fb048b4a7766d5',
  'f5a52ba78b2e4772a7315af44e0439e0',
  '2238391e894144e9aaf38aaa16433153'],
 ['ec8f8c12b70e4969ae24adb7031d9312',
  '3851376d4f84494ebdc080b34ddc0f5e',
  '087bf4ad4a0b4a17afc2d6c173eceff3',
  '99142bab4f864a50ba45e703c5b7f08d',
  '14c83bad01c34657ab846a7a2446e812'],
 ['ec8f8c12b70e4969ae24adb7031d9312',
  '38513

In [39]:
full_trajectories_seen_res[key]

[['ec8f8c12b70e4969ae24adb7031d9312',
  '3851376d4f84494ebdc080b34ddc0f5e',
  '558c07ff6d6d4d268f6c8de7abd76d1d',
  'cb6b9124a9594b4788966d28129cf531',
  'cb6b9124a9594b4788966d28129cf531',
  'cb6b9124a9594b4788966d28129cf531',
  'cb6b9124a9594b4788966d28129cf531',
  '090a8663beeb4ce6aeadfe720ea0c8e9',
  '090a8663beeb4ce6aeadfe720ea0c8e9',
  '090a8663beeb4ce6aeadfe720ea0c8e9',
  '090a8663beeb4ce6aeadfe720ea0c8e9',
  '090a8663beeb4ce6aeadfe720ea0c8e9',
  '4d76fa50a0e743e695fb048b4a7766d5',
  '4d76fa50a0e743e695fb048b4a7766d5',
  'ae8b1eb9dcd444688117d22839834a20',
  'fdd1efb5300a4a57b6537c492c27abe8',
  'fdd1efb5300a4a57b6537c492c27abe8',
  'fdd1efb5300a4a57b6537c492c27abe8',
  'fdd1efb5300a4a57b6537c492c27abe8',
  'fdd1efb5300a4a57b6537c492c27abe8',
  'fdd1efb5300a4a57b6537c492c27abe8'],
 ['ec8f8c12b70e4969ae24adb7031d9312',
  'ec8f8c12b70e4969ae24adb7031d9312',
  'ec8f8c12b70e4969ae24adb7031d9312',
  'ec8f8c12b70e4969ae24adb7031d9312',
  '3851376d4f84494ebdc080b34ddc0f5e',
  '087bf4ad